<a href="https://colab.research.google.com/github/GabeMaldonado/AIforMedicine/blob/master/AIforMed_W1_Theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes on AI for Medicine -- Week 1

The chest X-ray is one of the most common diagnostic imaging procedures in medicine with **~ 2 billion** chest X-rays taken a year. Chest X-ray interpretation is critical for detecting pneumonia, lung cancer, etc. 

An abonormality in chest X-rays is called a **mass**. A mass is defined as a **lision** or damaged tissue seen on a chest X-ray as greater than 3cm in diameter. 

During training, an algorithm is shown images of chest X-rays labeled with whether they contain a mass or not. The algorithm learns using these images and labels and it produces an output of whether the X-ray contains a mass. 
The algorithm can be a:
*   Deep Learning algorithm
*   Model
*   Convolutional Neural Network

The algorithm produces an output in terms of **scores** which are probabilities that the image contains a mass. 

## Key Challenges for Training Algorithms on Medical Images



1.   Class Imbalance
2.   Multi-Task
3.   Dataset Size


## Class Imbalance

There is not an equal number of examples of non-disease (normal) and disease (mass) in medical datasets. This is a reflection of the **prevalence** or the frequency of disease in the real-world where we see that there are far more examples of normals than of mass, especially when looking at X-rays of a healthy population. In a medical df we might see 100 times as many normal examples as mass examples. 
This creates a problem for the algo which would see mostly normal examples. This yields a model that starts to predict a very low probability of disease for everybody and won't be able to identify an example that has a disease. 





Let's say we have 8 images, 6 of which are normal and 2 are mass.
The value of the **loss function** is 0.3 for each.

Total Loss from Mass Examples   ```= 0.3 * 2 = 0.6```

Total Loss from Normal Examples ```= 0.3 * 6 = 1.8```

Most of the contribution to the loss comes from normal examples rather than from the mass examples, so the algo is optimizing its updates to get the normal examples right and it is not giving much relative weight to the mass examples. This will not produce a very good image classifier. **This is a class imbalance problem.**

The solution to this problem is to modify the loss function to weight the normal and the mass classes differently. We can assign weights to the positive and negative examples. $W_{p}$ for positive and $W_{n}$ for negative. In this case, we want to weight the mass examples more so they can have en equal contribution overall to the loss as the normal examples. Let's select ```6/8``` as the weight we have on the mass examples and ```2/8``` as the weight for the normal examples. 

$W_{p}$ or $W_{n}$ * $loss$

$W_{n} = 2/8 * 0.3 = 0.075 $

$W_{p} = 6/8 * 0.3 = 0.225$

Total Loss from Mass Examples   $= 0.225 * 2 = 0.45$

Total Loss from Normal Examples $= 0.075 * 6 = 0.45$

As we can see, if we sum up the total loss from the mass example we get 0.45 and this is equal to the total loss of the normal examples. 
In general, the weight we would put on the positive class will be the number of negative examples over the total number of examples.

$W_{p} = num negative / num total  = 6/8$ 6 normal examples / 8 (total) examples

$W_{n} = num positive / num total = 2/8$ 2 positive cases / 8 (total) cases

So now the positive and negative class contributions to the loss are the same. This is the idea of modifying the loss using weights and this method is called the **Weighted Loss** to tackle the class imbalance problem. 






### Weighted Loss Equation

- The loss is made up of two terms:
    - $loss_{pos}$: we'll use this to refer to the loss where the actual label is positive (the positive examples).
    - $loss_{neg}$: we'll use this to refer to the loss where the actual label is negative (the negative examples).  
- Note that within the $log()$ function, we'll add a tiny positive value, to avoid an error if taking the log of zero.

$$ loss^{(i)} = loss_{pos}^{(i)} + los_{neg}^{(i)} $$

$$loss_{pos}^{(i)} = -1 \times weight_{pos}^{(i)} \times y^{(i)} \times log(\hat{y}^{(i)} + \epsilon)$$

$$loss_{neg}^{(i)} = -1 \times weight_{neg}^{(i)} \times (1- y^{(i)}) \times log(1 - \hat{y}^{(i)} + \epsilon)$$

$$\epsilon = \text{a tiny positive number}$$